In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet
from cvcore.data.auto_augment import RandAugment
from PIL import Image

In [2]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/home/chec/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
HEIGHT = 137
WIDTH = 236

In [8]:
#import albumentations as albu
def get_train_augs():
    return RandAugment(n=2, m=27)

In [9]:
#plt.imshow(x)

In [10]:
#np.arange(10).copy()

In [11]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        orig_img = img.copy()
        #print(img.shape)
        if self.train_mode:
            augs = get_train_augs()
            #img = augs(image=img)['image']
            img = np.asarray(augs(Image.fromarray(img)))
        
        img = np.expand_dims(img, axis=-1)
        orig_img = np.expand_dims(orig_img, axis=-1)
        
        #print('###', img.shape)
        #img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        #MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        #STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        orig_img = transforms.functional.to_tensor(orig_img)
        
        #img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        elif self.train_mode:
            return img, orig_img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
                    
    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

    train_df = shuffle(train_df, random_state=1234)

    grapheme_words = np.unique(train_df.grapheme.values)
    grapheme_words_dict = {grapheme: i for i, grapheme in enumerate(grapheme_words)}
    train_df['word_label'] = train_df['grapheme'].map(lambda x: grapheme_words_dict[x])

    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [12]:
#train_loader, val_loader = get_train_val_loaders()

In [13]:
#for x in train_loader:
#    print(x)
#    break

# model

In [14]:
#import pretrainedmodels

In [15]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [16]:
from argparse import Namespace
import timm
from timm.models.activations import Swish, Mish
from timm.models.adaptive_avgmax_pool import SelectAdaptivePool2d

In [17]:
MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]

In [18]:
MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]
import timm
from timm.models.activations import Swish, Mish
from timm.models.adaptive_avgmax_pool import SelectAdaptivePool2d

class BengaliNet4(nn.Module):
    def __init__(self, backbone_name='se_resnext101_32x4d'):
        super(BengaliNet4, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.n_word = 1295
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant + self.n_word
        
        self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
        self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)
        
        self.num_p2_features = self.backbone.layer2[-1].se_module.fc2.out_channels
        self.num_p3_features = self.backbone.layer3[-1].se_module.fc2.out_channels
        self.p2_head = nn.Conv2d(self.num_p2_features, self.num_p2_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.p3_head = nn.Conv2d(self.num_p3_features, self.num_p3_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn2 = nn.BatchNorm2d(self.num_p2_features * 4)
        self.bn3 = nn.BatchNorm2d(self.num_p3_features * 4)
        self.act2 = Swish()
        self.act3 = Swish()
        
        self.fc_aux1 = nn.Linear(self.num_p3_features * 4, self.num_classes)
        self.fc_aux2 = nn.Linear(self.num_p2_features * 4, self.num_classes)
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        for fc in [self.fc, self.fc_aux1, self.fc_aux2]:
            nn.init.zeros_(fc.bias.data)

        print('init model4')
        
    def features(self, x):
        x = self.backbone.layer0(x); #print(x.size())
        x = self.backbone.layer1(x); #print(x.size())
        x = self.backbone.layer2(x); p2 = x; p2 = self.p2_head(p2); p2 = self.bn2(p2); p2 = self.act2(p2) #print(x.size())
        x = self.backbone.layer3(x); p3 = x; p3 = self.p3_head(p3); p3 = self.bn3(p3); p3 = self.act3(p3) #print(x.size())
        x = self.backbone.layer4(x); #print(x.size())
        return x, p2, p3
        
    def logits(self, x, p2, p3):
        x = self.avg_pool(x)
        #x = F.dropout2d(x, 0.2, self.training)
        x = torch.flatten(x, 1)
        
        p2 = self.avg_pool(p2)
        p2 = torch.flatten(p2, 1)
        
        p3 = self.avg_pool(p3)
        p3 = torch.flatten(p3, 1)
        return self.fc(x), self.fc_aux1(p3), self.fc_aux2(p2)
    
    def forward(self, x):
        x = F.interpolate(x, size=(224,224), mode='bilinear', align_corners=False)
        for i in range(len(x)):
            transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)
        x = torch.cat([x,x,x], 1)
        #x = self.conv0(x)
        #print(x.size())
        x, p2, p3 = self.features(x)
        x, logits_aux1, logits_aux2 = self.logits(x, p2, p3)

        return x, logits_aux1, logits_aux2

In [19]:
MODEL_DIR = './model4-ckps'
def create_model(args):
    model = BengaliNet4(args.backbone)
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [20]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [21]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [22]:
round(1/9, 6)

0.111111

In [23]:
import numpy as np
import sklearn.metrics
import torch


def calc_metrics(preds0, preds1, preds2, preds3, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2) == len(preds3)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    recall_word = sklearn.metrics.recall_score(preds3, y[:, 3], average='macro')
    
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    metrics['recall_word'] = round(recall_word, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    metrics['acc_word'] = round((preds3 == y[:, 3]).sum() / len(y), 6)    
    
    return metrics

In [24]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    loss3 = F.cross_entropy(outputs[3], y_true[:, 3], reduction='mean')
    
    return loss0 + loss1 + loss2 + loss3 #, loss0.item(), loss1.item(), loss2.item()
    #return loss3

In [25]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2, loss3 = 0., 0., 0., 0.
    preds0, preds1, preds2, preds3 = [], [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs, outputs_aux1, outputs_aux2 = model(x)
            #avg_outputs = torch.mean(torch.stack([outputs, outputs_aux1, outputs_aux2], 0), 0)
            outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            preds3.append(torch.max(outputs[3], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            loss3 += F.cross_entropy(outputs[3], y[:, 3], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    preds3 = torch.cat(preds3, 0).cpu().numpy()
    
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, preds3, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    metrics['loss_word'] = round(loss3 / val_loader.num, 6)
    
    return metrics
            

In [26]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [27]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [28]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [29]:
def rand_bbox_old(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [30]:
def rand_bbox(size, lam):
    H = size[2]
    W = size[3]

    x_margin_rate = 0.2

    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * (1-x_margin_rate*2) * cut_rat)
    cut_h = np.int(H * cut_rat)
    
    min_x_center = np.int(W * x_margin_rate + cut_w / 2)
    max_x_center = np.int(W * (1-x_margin_rate) - cut_w / 2)
    #print(min_x_center, max_x_center, lam, cut_w)
    min_y_center = cut_h // 2
    max_y_center = H - cut_h // 2
    if max_y_center == min_y_center:
        max_y_center += 1

    # uniform
    cx = np.random.randint(min_x_center, max_x_center)
    cy = np.random.randint(min_y_center, max_y_center)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    
    #print(bbx1, bbx2, bby1, bby2)

    return bbx1, bby1, bbx2, bby2

In [31]:
np.random.random()

0.9607663071143503

In [32]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam
from gridmask import GridMask

In [33]:
from cvcore.solver import WarmupCyclicalLR
def make_optimizer(model, base_lr=4e-4, weight_decay=0., weight_decay_bias=0., epsilon=1e-3):
    """
    Create optimizer with per-layer learning rate and weight decay.
    """
    params = []
    for key, value in model.named_parameters():
        if not value.requires_grad:
            continue
        lr = base_lr
        params += [{"params": [value], "lr": lr, "weight_decay": weight_decay_bias if 'bias' in key else weight_decay}]
    
    optimizer = torch.optim.AdamW(params, lr, eps=epsilon)
    return optimizer

In [34]:
best_metrics = 0.

def train(args, model):
    optimizer = make_optimizer(model)
    scheduler = WarmupCyclicalLR(
        "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)
    
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    for cycle in range(1, args.num_cycles+1):
        print('CYCLE:', cycle)
        train_cycle(args, model, optimizer, scheduler)
        #args.base_lr = 1e-4
        #args.num_epochs = 80
        args.warmup_epochs = 5
        scheduler = WarmupCyclicalLR(
            "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)

def train_cycle(args, model, optimizer, lr_scheduler):
    global best_metrics
    best_key = 'recall'
    
    val_metrics = validate(model, val_loader)
    print(val_metrics)

    if val_metrics[best_key] > best_metrics:
        best_metrics = val_metrics[best_key]
    
    model.train()
    train_iter = 0
    grid = GridMask(64, 128, rotate=15, ratio=0.6, mode=1, prob=1.)

    for epoch in range(args.num_epochs):
        grid.set_prob(epoch, args.st_epochs)

        train_loss = 0

        bg = time.time()
        for batch_idx, (img, orig_img, targets) in enumerate(train_loader):
            train_iter += 1
            img, orig_img, targets  = img.cuda(), orig_img.cuda(), targets.cuda()
            batch_size = img.size(0)
            r = np.random.rand()

            if r < 0.3:
                # generate mixed sample
                lam = np.random.beta(args.beta, args.beta)
                rand_index = torch.randperm(img.size()[0]).cuda()
                target_a = targets
                target_b = targets[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
                #img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                img[:, :, bby1:bby2, bbx1:bbx2] = img[rand_index, :, bby1:bby2, bbx1:bbx2]
                
                # adjust lambda to exactly match pixel ratio
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
                # compute output
                outputs, outputs_aux1, outputs_aux2 = model(img)
                loss_primary = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
                loss_aux1 = criterion(outputs_aux1, target_a) * lam + criterion(outputs_aux1, target_b) * (1. - lam)
                loss_aux2 = criterion(outputs_aux2, target_a) * lam + criterion(outputs_aux2, target_b) * (1. - lam)
                loss = loss_primary + loss_aux1*0.9 + loss_aux2*0.8
            elif r > 0.7:
                img = grid(img)
                outputs, outputs_aux1, outputs_aux2 = model(img)
                loss_primary = criterion(outputs, targets)
                loss_aux1 = criterion(outputs_aux1, targets)
                loss_aux2 = criterion(outputs_aux2, targets)
                loss = loss_primary + loss_aux1*0.9 + loss_aux2*0.8
            else:
                orig_img, targets = mixup(orig_img, targets)
                outputs, outputs_aux1, outputs_aux2 = model(orig_img)
                loss_primary = mixup_criterion(outputs, targets)
                loss_aux1 = mixup_criterion(outputs_aux1, targets)
                loss_aux2 = mixup_criterion(outputs_aux2, targets)
                loss = loss_primary + loss_aux1*0.9 + loss_aux2*0.8
                #loss = criterion(outputs, targets)

            optimizer.zero_grad()
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            #loss.backward()
            lr_scheduler(optimizer, batch_idx, epoch)
            optimizer.step()            
            
            current_lr = get_lrs(optimizer)

            train_loss += loss.item()
            print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
                epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
                loss.item(), train_loss/(batch_idx+1)), end='')

        if True:#train_iter > 0 and train_iter % args.iter_val == 0:
            val_metrics = validate(model, val_loader)
            print('\nval:', val_metrics)
                
            if val_metrics[best_key] > best_metrics:
                best_metrics = val_metrics[best_key]
                save_model(model, model_file)
                print('###>>>>> saved')
                
            model.train()

In [35]:
args = Namespace()
args.backbone = 'se_resnext101_32x4d'
args.ckp_name = 'model4_se_resnext101_fold3_224.pth'

args.base_lr = 2e-4
args.num_epochs = 80
args.warmup_epochs = 5
args.num_cycles = 100
args.batch_size = 440
args.val_batch_size = 1024
args.st_epochs = 5

args.beta = 1.0
args.cutmix_prob = 0.5

In [36]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=3)

(200840, 6)
(200840, 32332)
(160716, 6) (40124, 6)


In [37]:
model, model_file = create_model(args)
#if torch.cuda.device_count() > 1:
#    model = nn.DataParallel(model)
model = model.cuda()


init model4
model file: ./model4-ckps/se_resnext101_32x4d/model4_se_resnext101_fold3_224.pth, exist: True
loading ./model4-ckps/se_resnext101_32x4d/model4_se_resnext101_fold3_224.pth...


In [38]:
#for param in model.backbone.parameters():
#    param.requires_grad = False

In [39]:
#validate(nn.DataParallel(model), val_loader)

In [ ]:
train(args, model)

CYCLE: 1
{'recall': 0.974315, 'recall_grapheme': 0.957545, 'recall_vowel': 0.989539, 'recall_consonant': 0.992631, 'recall_word': 0.937249, 'acc_grapheme': 0.955638, 'acc_vowel': 0.989358, 'acc_consonant': 0.987663, 'acc_word': 0.92411, 'loss_grapheme': 0.204077, 'loss_vowel': 0.058991, 'loss_consonant': 0.056561, 'loss_word': 0.329869}
    0 | 0.000040 | 160600/160716 | 5.7291 | 10.5365 ||
val: {'recall': 0.989055, 'recall_grapheme': 0.984259, 'recall_vowel': 0.993084, 'recall_consonant': 0.994619, 'recall_word': 0.983246, 'acc_grapheme': 0.983526, 'acc_vowel': 0.994193, 'acc_consonant': 0.994068, 'acc_word': 0.983152, 'loss_grapheme': 0.068397, 'loss_vowel': 0.039506, 'loss_consonant': 0.033124, 'loss_word': 0.067768}
###>>>>> saved
    1 | 0.000080 | 160600/160716 | 1.7460 | 9.4575 |||
val: {'recall': 0.990077, 'recall_grapheme': 0.985977, 'recall_vowel': 0.993699, 'recall_consonant': 0.994656, 'recall_word': 0.985932, 'acc_grapheme': 0.98537, 'acc_vowel': 0.994517, 'acc_consonant':

   20 | 0.000168 | 160600/160716 | 8.0262 | 9.8860 |||
val: {'recall': 0.990934, 'recall_grapheme': 0.98712, 'recall_vowel': 0.99417, 'recall_consonant': 0.995324, 'recall_word': 0.986762, 'acc_grapheme': 0.986716, 'acc_vowel': 0.995015, 'acc_consonant': 0.995065, 'acc_word': 0.986517, 'loss_grapheme': 0.07838, 'loss_vowel': 0.060853, 'loss_consonant': 0.04413, 'loss_word': 0.068844}
   21 | 0.000165 | 160600/160716 | 1.6605 | 9.1677 |||
val: {'recall': 0.99013, 'recall_grapheme': 0.985327, 'recall_vowel': 0.99385, 'recall_consonant': 0.996014, 'recall_word': 0.985874, 'acc_grapheme': 0.985395, 'acc_vowel': 0.994891, 'acc_consonant': 0.99519, 'acc_word': 0.985495, 'loss_grapheme': 0.073322, 'loss_vowel': 0.049508, 'loss_consonant': 0.038459, 'loss_word': 0.066963}
   22 | 0.000162 | 160600/160716 | 24.4420 | 9.5924 ||
val: {'recall': 0.99141, 'recall_grapheme': 0.986719, 'recall_vowel': 0.995302, 'recall_consonant': 0.996901, 'recall_word': 0.987243, 'acc_grapheme': 0.986866, 'acc_vowe

###>>>>> saved
   41 | 0.000092 | 160600/160716 | 1.5070 | 8.6557 |||
val: {'recall': 0.992322, 'recall_grapheme': 0.988741, 'recall_vowel': 0.994258, 'recall_consonant': 0.997551, 'recall_word': 0.98757, 'acc_grapheme': 0.987937, 'acc_vowel': 0.995489, 'acc_consonant': 0.995813, 'acc_word': 0.98724, 'loss_grapheme': 0.050951, 'loss_vowel': 0.030896, 'loss_consonant': 0.026127, 'loss_word': 0.05112}
   42 | 0.000088 | 160600/160716 | 10.2968 | 8.5079 ||
val: {'recall': 0.991637, 'recall_grapheme': 0.987464, 'recall_vowel': 0.995304, 'recall_consonant': 0.996318, 'recall_word': 0.986679, 'acc_grapheme': 0.987489, 'acc_vowel': 0.995738, 'acc_consonant': 0.995614, 'acc_word': 0.986417, 'loss_grapheme': 0.057776, 'loss_vowel': 0.033966, 'loss_consonant': 0.028304, 'loss_word': 0.056438}
   43 | 0.000084 | 160600/160716 | 1.2920 | 9.1411 |||
val: {'recall': 0.989792, 'recall_grapheme': 0.984614, 'recall_vowel': 0.994327, 'recall_consonant': 0.995612, 'recall_word': 0.983806, 'acc_grapheme':

   62 | 0.000021 | 160600/160716 | 19.6101 | 8.4000 ||
val: {'recall': 0.994551, 'recall_grapheme': 0.991918, 'recall_vowel': 0.996319, 'recall_consonant': 0.998049, 'recall_word': 0.990766, 'acc_grapheme': 0.991053, 'acc_vowel': 0.996386, 'acc_consonant': 0.997109, 'acc_word': 0.990629, 'loss_grapheme': 0.033562, 'loss_vowel': 0.018168, 'loss_consonant': 0.014472, 'loss_word': 0.034619}
###>>>>> saved
   63 | 0.000019 | 160600/160716 | 1.3499 | 8.7183 |||
val: {'recall': 0.993533, 'recall_grapheme': 0.990477, 'recall_vowel': 0.99579, 'recall_consonant': 0.997389, 'recall_word': 0.98921, 'acc_grapheme': 0.989632, 'acc_vowel': 0.996137, 'acc_consonant': 0.996536, 'acc_word': 0.988934, 'loss_grapheme': 0.042751, 'loss_vowel': 0.025261, 'loss_consonant': 0.020895, 'loss_word': 0.043512}
   64 | 0.000017 | 160600/160716 | 1.5855 | 9.3146 |||
val: {'recall': 0.993643, 'recall_grapheme': 0.990681, 'recall_vowel': 0.99579, 'recall_consonant': 0.997421, 'recall_word': 0.988928, 'acc_grapheme':

    2 | 0.000119 | 160600/160716 | 6.2717 | 8.0934 |||
val: {'recall': 0.992794, 'recall_grapheme': 0.989417, 'recall_vowel': 0.995289, 'recall_consonant': 0.997052, 'recall_word': 0.987423, 'acc_grapheme': 0.988685, 'acc_vowel': 0.995913, 'acc_consonant': 0.996112, 'acc_word': 0.987314, 'loss_grapheme': 0.047483, 'loss_vowel': 0.025202, 'loss_consonant': 0.021114, 'loss_word': 0.050361}
    3 | 0.000159 | 160600/160716 | 4.0098 | 9.0679 ||
val: {'recall': 0.992112, 'recall_grapheme': 0.988294, 'recall_vowel': 0.995234, 'recall_consonant': 0.996625, 'recall_word': 0.98679, 'acc_grapheme': 0.987638, 'acc_vowel': 0.995738, 'acc_consonant': 0.995963, 'acc_word': 0.986567, 'loss_grapheme': 0.048227, 'loss_vowel': 0.021475, 'loss_consonant': 0.0195, 'loss_word': 0.05244}
    4 | 0.000198 | 160600/160716 | 2.0486 | 8.8183 |||
val: {'recall': 0.992865, 'recall_grapheme': 0.989807, 'recall_vowel': 0.995811, 'recall_consonant': 0.996036, 'recall_word': 0.988811, 'acc_grapheme': 0.988984, 'acc_v

   23 | 0.000159 | 160600/160716 | 1.5050 | 8.2598 |||
val: {'recall': 0.992994, 'recall_grapheme': 0.989738, 'recall_vowel': 0.995263, 'recall_consonant': 0.997235, 'recall_word': 0.989203, 'acc_grapheme': 0.989632, 'acc_vowel': 0.995838, 'acc_consonant': 0.996187, 'acc_word': 0.989059, 'loss_grapheme': 0.046067, 'loss_vowel': 0.032504, 'loss_consonant': 0.023884, 'loss_word': 0.043583}
   24 | 0.000156 | 160600/160716 | 2.8431 | 9.0982 |||
val: {'recall': 0.993058, 'recall_grapheme': 0.990005, 'recall_vowel': 0.995386, 'recall_consonant': 0.996837, 'recall_word': 0.988458, 'acc_grapheme': 0.988585, 'acc_vowel': 0.996087, 'acc_consonant': 0.996262, 'acc_word': 0.988236, 'loss_grapheme': 0.049604, 'loss_vowel': 0.028385, 'loss_consonant': 0.022866, 'loss_word': 0.048434}
   25 | 0.000152 | 160600/160716 | 22.0933 | 9.0264 |
val: {'recall': 0.991885, 'recall_grapheme': 0.988229, 'recall_vowel': 0.995104, 'recall_consonant': 0.995976, 'recall_word': 0.986251, 'acc_grapheme': 0.987539, 'a

In [40]:
train(args, model)

CYCLE: 1
{'recall': 0.971385, 'recall_grapheme': 0.958317, 'recall_vowel': 0.982632, 'recall_consonant': 0.986275, 'recall_word': 0.953239, 'acc_grapheme': 0.953843, 'acc_vowel': 0.987838, 'acc_consonant': 0.987339, 'acc_word': 0.949855, 'loss_grapheme': 0.253012, 'loss_vowel': 0.098051, 'loss_consonant': 0.079705, 'loss_word': 0.261843}
    0 | 0.000040 | 160640/160716 | 6.5186 | 13.2028 ||
val: {'recall': 0.979137, 'recall_grapheme': 0.969254, 'recall_vowel': 0.989295, 'recall_consonant': 0.988745, 'recall_word': 0.967247, 'acc_grapheme': 0.967451, 'acc_vowel': 0.99018, 'acc_consonant': 0.990729, 'acc_word': 0.966529, 'loss_grapheme': 0.1693, 'loss_vowel': 0.080292, 'loss_consonant': 0.065291, 'loss_word': 0.16706}
###>>>>> saved
    1 | 0.000080 | 160640/160716 | 3.1096 | 13.1037 ||
val: {'recall': 0.98112, 'recall_grapheme': 0.971663, 'recall_vowel': 0.99, 'recall_consonant': 0.991156, 'recall_word': 0.970476, 'acc_grapheme': 0.970093, 'acc_vowel': 0.990928, 'acc_consonant': 0.9911

###>>>>> saved
   20 | 0.000336 | 160640/160716 | 8.1848 | 9.8692 |||
val: {'recall': 0.984405, 'recall_grapheme': 0.977018, 'recall_vowel': 0.991063, 'recall_consonant': 0.992519, 'recall_word': 0.976616, 'acc_grapheme': 0.975775, 'acc_vowel': 0.992299, 'acc_consonant': 0.992374, 'acc_word': 0.976124, 'loss_grapheme': 0.117133, 'loss_vowel': 0.06618, 'loss_consonant': 0.054526, 'loss_word': 0.10982}
   21 | 0.000330 | 160640/160716 | 13.9724 | 11.1551 |
val: {'recall': 0.983682, 'recall_grapheme': 0.974904, 'recall_vowel': 0.992285, 'recall_consonant': 0.992636, 'recall_word': 0.974476, 'acc_grapheme': 0.974903, 'acc_vowel': 0.993071, 'acc_consonant': 0.992174, 'acc_word': 0.973781, 'loss_grapheme': 0.116202, 'loss_vowel': 0.063143, 'loss_consonant': 0.052422, 'loss_word': 0.112797}
   22 | 0.000324 | 160640/160716 | 26.0440 | 10.5056 |
val: {'recall': 0.985931, 'recall_grapheme': 0.978673, 'recall_vowel': 0.992499, 'recall_consonant': 0.993877, 'recall_word': 0.979078, 'acc_grapheme'

   41 | 0.000184 | 160640/160716 | 2.1980 | 9.8575 |||
val: {'recall': 0.988541, 'recall_grapheme': 0.982392, 'recall_vowel': 0.993822, 'recall_consonant': 0.995561, 'recall_word': 0.983101, 'acc_grapheme': 0.982255, 'acc_vowel': 0.993919, 'acc_consonant': 0.994268, 'acc_word': 0.982828, 'loss_grapheme': 0.073723, 'loss_vowel': 0.041669, 'loss_consonant': 0.033414, 'loss_word': 0.068638}
   42 | 0.000177 | 160640/160716 | 12.1496 | 10.0625 |
val: {'recall': 0.988676, 'recall_grapheme': 0.983335, 'recall_vowel': 0.992984, 'recall_consonant': 0.995049, 'recall_word': 0.982107, 'acc_grapheme': 0.981283, 'acc_vowel': 0.994093, 'acc_consonant': 0.993919, 'acc_word': 0.981831, 'loss_grapheme': 0.098801, 'loss_vowel': 0.061308, 'loss_consonant': 0.053519, 'loss_word': 0.08689}
   43 | 0.000169 | 160640/160716 | 12.7108 | 10.4040 |
val: {'recall': 0.989115, 'recall_grapheme': 0.984596, 'recall_vowel': 0.993529, 'recall_consonant': 0.993739, 'recall_word': 0.984719, 'acc_grapheme': 0.98385, 'ac

###>>>>> saved
   62 | 0.000043 | 160640/160716 | 14.2375 | 9.8167 ||
val: {'recall': 0.990957, 'recall_grapheme': 0.987066, 'recall_vowel': 0.994248, 'recall_consonant': 0.995446, 'recall_word': 0.985634, 'acc_grapheme': 0.985545, 'acc_vowel': 0.995015, 'acc_consonant': 0.994841, 'acc_word': 0.985495, 'loss_grapheme': 0.061837, 'loss_vowel': 0.035317, 'loss_consonant': 0.029587, 'loss_word': 0.058413}
   63 | 0.000038 | 160640/160716 | 2.2644 | 8.7743 |||
val: {'recall': 0.990761, 'recall_grapheme': 0.987458, 'recall_vowel': 0.994194, 'recall_consonant': 0.993933, 'recall_word': 0.986092, 'acc_grapheme': 0.985719, 'acc_vowel': 0.994841, 'acc_consonant': 0.994916, 'acc_word': 0.986018, 'loss_grapheme': 0.065662, 'loss_vowel': 0.04173, 'loss_consonant': 0.033715, 'loss_word': 0.058626}
   64 | 0.000034 | 160640/160716 | 1.5652 | 9.3138 ||
val: {'recall': 0.990578, 'recall_grapheme': 0.986867, 'recall_vowel': 0.994158, 'recall_consonant': 0.99442, 'recall_word': 0.986565, 'acc_grapheme':

    2 | 0.000119 | 160640/160716 | 1.3608 | 9.7764 |||
val: {'recall': 0.990208, 'recall_grapheme': 0.985477, 'recall_vowel': 0.99368, 'recall_consonant': 0.996198, 'recall_word': 0.984568, 'acc_grapheme': 0.984049, 'acc_vowel': 0.994318, 'acc_consonant': 0.994467, 'acc_word': 0.984373, 'loss_grapheme': 0.097951, 'loss_vowel': 0.075754, 'loss_consonant': 0.055121, 'loss_word': 0.079259}
    3 | 0.000159 | 160640/160716 | 1.9220 | 8.7770 |||
val: {'recall': 0.990753, 'recall_grapheme': 0.986459, 'recall_vowel': 0.994259, 'recall_consonant': 0.995835, 'recall_word': 0.984751, 'acc_grapheme': 0.984573, 'acc_vowel': 0.994841, 'acc_consonant': 0.994791, 'acc_word': 0.984598, 'loss_grapheme': 0.058571, 'loss_vowel': 0.028828, 'loss_consonant': 0.025588, 'loss_word': 0.058152}
    4 | 0.000198 | 160640/160716 | 11.5189 | 8.8218 |
val: {'recall': 0.98848, 'recall_grapheme': 0.982883, 'recall_vowel': 0.993702, 'recall_consonant': 0.994454, 'recall_word': 0.982399, 'acc_grapheme': 0.982454, 'acc

KeyboardInterrupt: 

In [ ]:
#save_model(model, model_file)

In [ ]:
del model